# Taller Clustering

**Objetivos del Taller**

- Entender los conceptos básicos de clustering (agrupamiento no supervisado) y su aplicación en finanzas.
- Aprender a utilizar los algoritmos K-Means (K-medias) y clustering jerárquico para agrupar fondos de inversión según características cuantitativas.
- Aprender a utilizar la herramienta GraphExt (https://www.graphext.com).
- Practicar la limpieza y preprocesamiento de datos financieros (precios históricos de fondos).
- Extraer características cuantitativas de los activos y comprender su significado financiero.
- Aplicar PCA y K-Means sobre estas características para identificar clústeres de fondos similares.
- Visualizar los resultados de clustering para interpretar los grupos.
- Interpretación financiera de los clústeres y discutir los resultados.

**Estructura del Taller**

- Presentación: Presentación de objetivos, repaso de aprendizaje no supervisado vs. supervisado, importancia del clustering en análisis financiero.
- Limpieza y preprocesamiento de datos: Carga del dataset de precios históricos de fondos, identificación de datos faltantes o anómalos, y técnicas de limpieza.
- Ingeniería de características: Cálculo de variables cuantitativas y preparación de la matriz de características para clustering.
- Aplicación de PCA (Análisis de Componentes Principales): Aplicación para reducción de dimensionalidad.
- Algoritmo K-Means, GraphExt y HAC: Clustering con distintos algoritmos.
- Discusión de resultados y conclusiones: Puesta en común de hallazgos, interpretación de clústeres, recomendaciones finales y cierre.

**Tesis Financiera**

Como gestor, creemos que el mercado asiático va a tener un buen comportamiento el próximo año, por lo tanto, queremos posicionar nuestro fondo de fondos con un claro sesgo a este mercado.

**Dataset**

- Valores liquidativos de 25.000 fondos de inversión entre 2016-01-05 y 2021-07-16 proporcionados por IronIA.
- Factores de Fama & French (Mkt-RF, SMB, HML, MOM) de Asia Pacífico ex Japón.
- Valores liquidativos de *iShares MSCI All Country Asia ex Japan ETF (AAXJ)*. **Queda prohibido su uso para extracción de características**. Se utilizará solamente como TEST al final del ejercicio.


## Lectura de datos

### NAV Fondos

In [ ]:
import pickle as pkl
import pandas as pd
import numpy as np
import missingno as mso
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Leemos los datos en crudo del pickle.
raw_data = pkl.load(open("./dataset/navs.pickle", "rb"))

In [ ]:
# TODO: Procesa el objeto para obtener los valores liquidativos de los fondos.

### Factores Fama & French

Ref.: https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html

In [ ]:
data_asia_pacific_ex_japan_3_factors_daily = pd.read_csv(
    "./dataset/Asia_Pacific_ex_Japan_3_Factors_Daily.csv",
    index_col="Date",
    parse_dates=True,
)

In [ ]:
data_asia_pacific_ex_japan_MOM_factor_daily = pd.read_csv(
    "./dataset/Asia_Pacific_ex_Japan_MOM_Factor_Daily.csv",
    index_col="Date",
    parse_dates=True,
)
data_asia_pacific_ex_japan_MOM_factor_daily.columns = ["MOM"]

In [ ]:
# TODO: Examina y procesa los factores de Fama y French.

## Limpieza de datos

In [ ]:
# TODO: Efectúa la limpieza y alinea los datos.

## Filtrado de fondos Asia Pacific ex-Japan

In [ ]:
import statsmodels.api as sm


def run_ordinary_least_squares(X: pd.Series, Y: pd.Series) -> tuple:
    """Ejecuta una regresión lineal y devuelve el modelo de regresión.

    Args:
        X (pd.Series): Variable independiente.
        Y (pd.Series): Variable dependiente

    Returns:
        Modelo de regresión.
    """
    X = sm.add_constant(X)

    model = sm.OLS(Y, X).fit()

    return model

In [ ]:
# TODO: Prepara los datos y filtra los fondos que nos interesan.

## Extracción de características

* **Exceso de rendimiento del fondo** respecto del mercado asiático de referencia ([Fondo-Rf] - ["Mkt-Rf]) a 1, 3 y 5 años.
* **Desviación estándar** a 1, 3 y 5 años.
* **Up-Market Capture Ratio**  a 1, 3 y 5 años: Mide el rendimiento relativo a un índice durante los mercados alcistas.
$ \text{Up-Market Capture Ratio}=\frac{\text{Portfolio return during Up-Market periods}}{\text{Benchmark return during Up-Market periods}} \times 100 $
* **Down-Market Capture Ratio**  a 1, 3 y 5 años: Mide el rendimiento relativo a un índice durante los mercados bajistas.
$ \text{Down-Market Capture Ratio}=\frac{\text{Portfolio return during Down-Market periods}}{\text{Benchmark return during Down-Market periods}} \times 100 $
* **Beta** de las regresiones con los 4 factores a 1, 3 y 5 años.

In [ ]:
def calcular_exceso_rentabilidad(funds: pd.DataFrame, market: pd.Series) -> pd.DataFrame:
    """Calcula la diferencia acumulada entre "funds" y "market".

    Args:
        funds (pd.DataFrame): Retornos de los fondos.
        market (pd.Series): Retornos del mercado.

    Returns:
        pd.DataFrame: Diferencia acumulada por fondo.
    """
    diff = funds.subtract(market, axis=0)
    diff = diff.sum(axis=0)

    return diff

In [ ]:
def calcular_desviacion_standar(funds: pd.DataFrame) -> pd.DataFrame:
    """Calcula la desviación estándar de los fondos.

    Args:
        funds (pd.DataFrame): Retornos de los fondos.

    Returns:
        pd.DataFrame: Desviación estándar por fondo.
    """
    std = funds.std(axis=0)

    return std

In [ ]:
def calcular_up_market_capture_ratio(funds: pd.DataFrame, market: pd.Series) -> pd.DataFrame:
    """Calcula el up market capture ratio de los fondos.

    Args:
        funds (pd.DataFrame): Retornos de los fondos.
        market (pd.Series): Retornos del mercado.

    Returns:
        pd.DataFrame: Up market capture ratio por fondo.
    """
    # Filtramos los días donde el los fondos suben.
    funds_up_days_mask = funds > 0
    funds_up_days_mask = funds_up_days_mask.astype(int)
    funds_up_days = funds * funds_up_days_mask
    funds_up_days = funds_up_days.sum(axis=0)

    # Filtramos los días donde el mercado sube.
    market_up_days_mask = market > 0
    market_up_days_mask = market_up_days_mask.astype(int)
    market_up_days = market * market_up_days_mask
    market_up_days = market_up_days.sum(axis=0)

    # Calculamos el up market capture ratio.
    ratio = funds_up_days / market_up_days

    return ratio

In [ ]:
def calcular_down_market_capture_ratio(funds: pd.DataFrame, market: pd.Series) -> pd.DataFrame:
    """Calcula el down market capture ratio de los fondos.

    Args:
        funds (pd.DataFrame): Retornos de los fondos.
        market (pd.Series): Retornos del mercado.

    Returns:
        pd.DataFrame: Down market capture ratio por fondo.
    """
    # Filtramos los días donde el los fondos bajan.
    funds_down_days_mask = funds < 0
    funds_down_days_mask = funds_down_days_mask.astype(int)
    funds_down_days = funds * funds_down_days_mask
    funds_down_days = funds_down_days.sum(axis=0)

    # Filtramos los días donde el mercado baja.
    market_down_days_mask = market < 0
    market_down_days_mask = market_down_days_mask.astype(int)
    market_down_days = market * market_down_days_mask
    market_down_days = market_down_days.sum(axis=0)

    # Calculamos el down market capture ratio.
    ratio = funds_down_days / market_down_days

    return ratio

In [ ]:
def calcular_regresion_lineal(fund: pd.Series, factor: pd.Series) -> dict:
    """Calcula la regresión lineal entre un fondo y un factor.

    Args:
        fund (pd.Series): Retornos del fondo
        factor (pd.Series): Factor de riesgo

    Returns:
        dict: Diccionario con los resultados de la regresión.
    """
    factor_name = factor.name
    model = run_ordinary_least_squares(factor, fund)

    # Extraemos los resultados de la regresión.
    alpha = model.params["const"]
    beta = model.params[factor.name]
    r2 = model.rsquared
    alpha_pvalue = model.pvalues["const"]
    beta_pvalue = model.pvalues[factor.name]
    alpha_tvalue = model.tvalues["const"]
    beta_tvalue = model.tvalues[factor.name]
    alpha_tvalue_adjusted = alpha * (1 - np.exp(-1 * np.abs(alpha_tvalue)))
    beta_tvalue_adjusted = beta * (1 - np.exp(-1 * np.abs(beta_tvalue)))

    # Guardamos los resultados en un diccionario.
    results = {
        f"{factor_name}_Alpha": alpha,
        f"{factor_name}_Beta": beta,
        f"{factor_name}_R2": r2,
        f"{factor_name}_Alpha p-value": alpha_pvalue,
        f"{factor_name}_Beta p-value": beta_pvalue,
        f"{factor_name}_Alpha t-value": alpha_tvalue,
        f"{factor_name}_Beta t-value": beta_tvalue,
        f"{factor_name}_Alpha t-value adjusted": alpha_tvalue_adjusted,
        f"{factor_name}_Beta t-value adjusted": beta_tvalue_adjusted,
    }
    return results

In [ ]:
# TODO: Realizar extracción de características para todos los fondos.

## PCA + Clustering con KMeans

In [ ]:
# TODO: Realiza PCA para reducir la dimensionalidad de los datos.
# TODO: Interpreta autovectores y autovalores. Selecciona el número de componentes principales.
# TODO: Realiza la proyección de los datos en el espacio de los componentes principales.
# TODO: Realiza clustering con KMeans (selecciona el k adecuado) en los datos proyectados.
# TODO: Examina e interpreta los centroides de los clusters.
# TODO: Selecciona el cluster que más te interese y filtra los fondos.
# TODO: Descarga los datos del ETF AAXJ y compara su comportamiento con los fondos seleccionados.

## Clustering con GraphExt

In [ ]:
# TODO: Prepara los datos para exportar a GraphExt.
# TODO: Realiza clustering de los fondos seleccionados usando k-NNG y UMAP.
# TODO: Visualiza los datos de los distintos clusters.

## Clustering Jerárquico Aglomerativo

In [ ]:
# TODO: Realiza clustering de los fondos seleccionados usando HAC.
# TODO: Visualiza los datos de los distintos clusters.


In [ ]:
# TODO: ¿Próximos pasos?